In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

Первое, что я сделала - загрузила данные с сайта "eda.ru" (код представлен в отдельном файле food.py ) В дальнейшем мы будем работать с датафреймом, составленным по этим данным. 

In [ ]:
recipes = pd.read_csv('food.csv')

In [ ]:
recipes.head(10)

Первое, что пришло в голову, это посмотреть, какие категории еды чаще всего встречаются на этом сайте. 

In [ ]:
category_data = recipes['категория'].value_counts()
category_data.head(10)

In [ ]:
category_data = category_data.to_frame()
categories = category_data.index.tolist()
counter = category_data['count'].tolist()
category_data.plot.pie(figsize=(8, 8), subplots=True)


In [ ]:
most_popular_category = category_data[category_data['count'] == max(counter)].index.tolist()[0]
least_popular_category = category_data[category_data['count'] == min(counter)].index.tolist()[0]
print(f'Больше всего рецептов из категори {most_popular_category}, меньше всего из {least_popular_category}')

Теперь давайте посмотрим, какая кухня чаще всего встречается

In [ ]:
cuisine_data = recipes['кухня'].value_counts()
cuisine_data.head(10)

In [ ]:
cuisine_data = cuisine_data.to_frame()
cuisine = cuisine_data.index.tolist()
counter = cuisine_data['count'].tolist()
category_data.plot.pie(figsize=(8, 8), subplots=True)

In [ ]:
most_popular_cuisine = cuisine_data[cuisine_data['count'] == max(counter)].index.tolist()[0]
least_popular_cuisine = cuisine_data[cuisine_data['count'] == min(counter)].index.tolist()[0]
print(f'Наиболее популярна {most_popular_cuisine}, наименее - {least_popular_cuisine}')

Теперь можно приступить к более детальному анализу

Я хочу проверить гипотезу о том, что есть отрицательная зависимость между количеством ингредиентов и количеством сохранений.Зачем тратить много денег на ингредиенты, если за эту же сумму можно заказать вкусное готовое блюдо. Для проверки гипотезы я сначала построю матрицу корреляции. Для этого нужно перевести некоторые значения в целочисленные 

In [ ]:
prepare_time = recipes['время приготовления'].to_list()
int_prepare_time = list()
for item in prepare_time:
    lst = item.split()
    if len(lst) == 2:
        if 'минут' in lst[1]:
            item = int(lst[0])
            int_prepare_time.append(item)
        elif 'час' in lst[1]: # переводим часы в минуты
            item = int(lst[0]) * 60
            int_prepare_time.append(item)
        elif 'сут' in lst[1]: # переводим сутки в минуты
            item = int(lst[0]) * 24 * 60
            int_prepare_time.append(item)
    elif len(lst) == 4:
        item = int(lst[0].strip()) * 60 + int(lst[2].strip())
        int_prepare_time.append(item)

ingredients_count = recipes['кол-во ингредиентов'].to_list()
int_ingredients_count = list()
for item in ingredients_count: # переводим значения из столбца "кол-во ингредиентов" в целочисленные
    lst = item.split()
    int_ingredients_count.append(int(lst[0]))
    
portions_count = recipes['кол-во порций'].to_list()
int_portions_count = list()
for item in ingredients_count: # переводим значения из столбца "кол-во порций" в целочисленные
    lst = item.split()
    int_portions_count.append(int(lst[0]))
    
save_count = recipes['кол-во сохранений'].to_list()# Сохраняем количества сохранений в отдельный список 
like_count = recipes['кол-во лайков'].to_list() #Аналогичное делаем с количеством лайков 
dislike_count = recipes['кол-во дизлайков'].to_list() #И с количеством дизлайков
 
#Сделаем новый датафрейм для удобства
data = {'кол-ва сохранений' : save_count, 'кол-во лайков' : like_count, 'кол-во дизлайков' : dislike_count, 'кол-во порций' : int_portions_count, 'кол-во ингредиентов' : int_ingredients_count, 'время приготовления' : int_prepare_time}
df = pd.DataFrame(data, columns=['кол-ва сохранений','кол-во лайков', 'кол-во дизлайков', 'кол-во порций', 'кол-во ингредиентов', 'время приготовления'])
df.tail(5)

Теперь все готово, чтобы строить корреляционную матрицу!

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

Из матрицы корреляции можно сделать много интересных выводов! Например, видно,что моя гипотеза подтвердилась! Действительно, между количеством ингредиентов и количеством сохранений есть отрицательная зависимость, однако совсем незначительная. 
 Также можно заметить, что между количеством лайков и дизлайков положительная зависимость, что логично. Чем больше людей посмотрели рецепт, тем большему количеству он как понравится, так и не понравится. Еще видно, что люди не любят тратить много времени на готовку и я не могу не согласиться. 

Теперь я построю график, связывающий количество сохранений и количество ингредиентов, чтобы наглядно продемонстрировать зависимость. 


Сначала отсортируем датафрейм по возрастанию сохранений


In [ ]:
df = df.sort_values(by='кол-ва сохранений')
print(df.tail(5))    

Итак, все готово, чтобы строить график

In [ ]:
#Строим график
fig, ax = plt.subplots(figsize=(20, 10))
plt.plot(df['кол-ва сохранений'], df['кол-во ингредиентов'])
plt.xticks(rotation=45)
plt.grid(True)
plt.xlabel('кол-ва сохранений', fontsize=20)
plt.ylabel('кол-во ингредиентов', fontsize=20)
plt.title('Зависимость между количеством сохранений и количеством ингредиентов', fontsize=20) 
#Посчитаем линию тренда
z = np.polyfit (df['кол-ва сохранений'], df['кол-во ингредиентов'], 1 )
p = np.poly1d (z)
#Добавим линию тренда
plt.plot (df['кол-ва сохранений'], p(df['кол-ва сохранений'])) 
plt.show()


Таким образом, линия тренда показывает отрицательную зависимость, что с увеличением кол-ва сохранений кол-во ингредиентов падает!

Теперь давате найдем кухню с "наилучшим" соотношением лайков и дизлайков . Моя гипотеза в том, что это просто обязана быть итальянская кухня.

Убираем пробелы из названий для работы с sql

In [ ]:
recipes = recipes.rename(columns={'вид меню': 'вид_меню', 'кол-во ингредиентов': 'кол-во_ингредиентов', 'кол-во лайков': 'лайки', 'кол-во дизлайков': 'дизлайки'})
recipes.head(10)

In [ ]:
engine = create_engine('sqlite:///recipes_db.sqlite')

recipes.to_sql('recipes', engine, if_exists='replace')

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///recipes_db.sqlite

In [ ]:
%%sql
WITH help as (SELECT кухня, AVG(лайки/дизлайки) AS Среднее_отношение_лайков_к_дизлайкам
FROM recipes
GROUP BY 1)

SELECT кухня, MAX(Среднее_отношение_лайков_к_дизлайкам) FROM help
;

Получили неожиданный результат! Видимо, стоит попробовать норвежскую кухню)

Напоследок давайте выведем самое вкусное блюдо на основе данных

In [ ]:
%%sql
WITH help as (SELECT название, кухня, AVG(лайки/дизлайки) AS Среднее_отношение_лайков_к_дизлайкам
FROM recipes
GROUP BY 1)

SELECT название,кухня, MAX(Среднее_отношение_лайков_к_дизлайкам) FROM help
;

Звучит очень вкусно!

На этом я завершаю свой проект. Всем приятного аппетита!